In [1]:
import re, random, os

import numpy as np
import pandas as pd

import nltk
import re 
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
stopWords_nltk = set(stopwords.words('english'))

import pandas as pd
import seaborn as sns
import re
import string
from collections import Counter, defaultdict

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 10
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import warnings
warnings.filterwarnings("ignore")

import nltk
import re
import string
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv("DisneylandReviews.csv")
df.drop(['Review_ID', 'Year_Month', 'Reviewer_Location'], axis=1, inplace=True)
df.rename(columns={'Rating': 'Rating', 'Review_Text': 'Review'}, inplace=True)
# df = df.iloc[:10000,:]
df['original_review'] = df['Review'].values
df.shape

(42656, 4)

In [3]:
from textacy.preprocessing.replace import urls, hashtags, numbers, emails, emojis, currency_symbols

In [4]:
# Remove duplicate values
df.drop_duplicates(keep=False, inplace=True)

# Remove punctuation
from textacy.preprocessing.remove import punctuation
df['Review'] = df['Review'].apply(punctuation)

# Convert to lowercase 
df['Review'] = df['Review'].map(lambda x: x.lower())

# Replace common entities
from textacy.preprocessing.replace import urls, hashtags, numbers, emails, emojis, currency_symbols
df['Review'] = df['Review'].\
 apply(urls).\
 apply(hashtags).\
 apply(currency_symbols).\
 apply(emojis).\
 apply(emails)

#Define and remove stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 
                   'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see',
                   'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 
                   'right', 'line', 'even', 'also', 'may', 'take', 'come'])
df['Review'] = df['Review'].apply(lambda x: " ".join(x for x in str(x).split() if x not in stop_words))

# Clean out Disney location references and ride references
df['Review'] = df['Review'].str.replace(r"disney(?:land| park)?|park|place|hong kong|hk|cali|california|paris|pa|fr|Disneyland Railroad|\
                            Great Moments with Mr. Lincoln | Lincoln| The Disney Gallery | Gallery| Main Street Vehicles | \
                            Main Street Cinema | Main Street | Walt Disney's Enchanted Tiki Room | Enchanted Tiki Room | Jungle Cruise | \
                            Indiana Jones | Pirates of the Caribbean | Haunted Mansion | \
                            Davy Crockett Explorer Canoes | Davy Crockett | Explorer Canoes | Many Adventures of Winnie the Pooh | Winnie the Pooh | Splash Mountain | \
                            Big Thunder Mountain Railroad | Mark Twain Riverboat | Mark Twain | Frontierland Shootin Exposition |Frontierland| \
                            Pirate's Lair on Tom Sawyer Island | Pirate's Lair | Tom Sawyer | Sailing Ship Columbia | \
                            Mickey's House | Meet Mickey | Minnie's House | Roger Rabbit's Car Toon Spin | Roger Rabbit | \
                            Chip 'n' Dale's Gadget Coaster | Gadget Coaster | Fantasyland Theatre | It's a Small World | Small World | \
                            Mr. Toad's Wild Ride | Peter Pan's Flight | Pinocchio's Daring Journey | Pixie Hollow |\
                            Snow White's Enchanted Wish | Snow White | Enchanted Wish | Storybook Land Canal Boats | Storybook Land| Canal Boats |Dumbo the Flying Elephant | Dumbo |\
                            Casey Jr. Circus Train | Circus Train | Alice in Wonderland | Matterhorn Bobsleds | Bobsleds | King Arthur Carrousel | King Arthur |\
                            Mad Tea Party | Sleeping Beauty Castle Walkthrough | Sleeping Beauty| Astro Orbiter | \
                            Autopia Star Tours – The Adventures Continue | Autopia Star Tours | Buzz Lightyear Astro Blasters | Buzz Lightyear | Astro Blasters |\
                            Space Mountain | Monorail | Finding Nemo Submarine Voyage | Finding Nemo | Submarine Voyage |\
                            Star Wars Launch Bay | Star Wars | Millennium Falcon: Smuggler's Run | Millennium Falcon | Smuggler's Run| Rise of the Resistance | \
                            Electrical Parade | Galaxy's Edge | Tomorrowland | Fantasyland | \
                            Mickey's Toontown | Toontown | Frontierland | Critter Country | New Orleans Square | Adventureland | \
                            Main Street, U.S.A.| toy story | lion king | main street | small world | sce mountain | \
                            story land | king show | mickey mouse | grizzly gulch | mystic manor | \
                            big thunder mountain | chinese new year | slinky dog ride | jungle river cruise | lion king show | \
                            toy story| festival lion king | runaway mine cars | mine cars | iron man |\
                            big grizzly mountain | sunny bay station | small world | mountain runaway mine | \
                            iron man experience | sleeping beauty castle | sce mountain | buzz light year | \
                            big thunder | iron man experience | mickey | minnie | \
                            big grizzley | buzz lightyear | mountain",'')

# Drop words with less than 5 frequencies
temp_df = pd.Series(' '.join(df['Review']).split()).value_counts()
drops = temp_df[temp_df <= 5]
df['Review'] = df['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in drops))



#Regex (parse out and normalize all references to people)
PERSON_PATTERN: re.Pattern = re.compile(r"\b(?:Great-grandfather'?s?\b|Great-grandmother'?s?\b|Great-uncle'?s?\b|\
                                        Grandfather'?s?\b|Grandmother'?s?\b|Great-aunt'?s?\b|Uncle|Aunt'?s?\b|Father'?s?\b|\
                                        Mother'?s?\b|Uncle'?s?\b|Sister'?s?\b|Brother-in-law'?s?\b|Brother'?s?\b|\
                                        Sister-in-law'?s?\b|Husband'?s?\b|Wife'?s?\b|Cousin'?s?\b|Cousins wife'?s?\b|\
                                        Cousin'?s?\b|Cousin'?s?\b husband'?s?\b|Nephew'?s?\b|Niece'?s?\b|Son'?s?\b|\
                                        Daughter-in-law'?s?\b|Daughter'?s?\b|Son-in-law'?s?\b|First cousin once removed'?s?\b|\
                                        grandson'?s?\b|grandson'?s?\b|grand\sson'?s?\b|grand\sdaughter'?s?\b|Granddaughter'?s?\b|Baby'?s?\b|Child'?s?\b|Infants'?s?\b|Newborn'?s?\b|\
                                        Mom'?s?\b|Dad'?s?\b|\d{0,3}\s?years?\sold'?s?\b|\d{0,3}\s?month'?s?\sold'?s?\b|\
                                        \d{0,3}\s?yr'?s?\sold'?s?\b|kid|kids|yr old|year old|u|pa|Mom'?s?\b|Dad'?s?\b|honey'?s?\b|\
                                        hubby'?s?\b|papa'?s?\b|mama'?s?\b|gram'?s?\b|gramp'?s?\b)\b", flags=re.IGNORECASE)
result = []
for value in df["Review"]:
    result.append(re.sub(PERSON_PATTERN, '_PERSON_', value))
df["Reviews"] = result 
df = df.drop('Review', axis=1)

In [5]:
# Negative reviews only for topic modeling
df = df[df['Rating']<=3].reset_index(drop=True)

In [6]:
nlp = spacy.load('en_core_web_md', disable = ['ner', 'parser'])

In [7]:
rows=[]
temp_list=[]
for review in df['Reviews'].values:
    doc = nlp(review)
    for token in doc:
        temp_list.append(token.lemma_)
    rows.append(' '.join(temp_list))
    temp_list=[]
    
df['lemma'] = rows

In [8]:
df['lemma'] = df['lemma'].str.replace(' _?person_? ', ' ', case=False)

In [25]:
vectorizer = TfidfVectorizer(ngram_range=(4,4), max_df=0.5, min_df=2, stop_words='english')
X = vectorizer.fit_transform(list(df['lemma'].values))
data = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
data.shape

(8729, 2970)

In [26]:
from sklearn.decomposition import NMF

In [27]:
nmf = NMF(n_components=4)
W = nmf.fit_transform(X)
H = nmf.components_
print(f"Original shape of X is {X.shape}")
print(f"Decomposed W matrix is {W.shape}")
print(f"Decomposed H matrix is {H.shape}")

Original shape of X is (8729, 2970)
Decomposed W matrix is (8729, 4)
Decomposed H matrix is (4, 2970)


In [28]:
from typing import List
import numpy as np
def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5, n_components: int = 3):
    nmf = NMF(n_components=n_components)
    W = nmf.fit_transform(X)
    H = nmf.components_
    for topic, vector in enumerate(H):
        print(f"TOPIC {topic}\n")
        total = vector.sum()
        top_scores = vector.argsort()[::-1][:num_top_tokens]
        token_names = list(map(lambda idx: feature_names[idx], top_scores))
        strengths = list(map(lambda idx: vector[idx] / total, top_scores))
    
        for strength, token_name in zip(strengths, token_names):
            print(f"{token_name} ({round(strength * 100, 1)}%)\n")
        print(f"=" * 50)

In [29]:
get_top_tf_idf_tokens_for_topic(H, data.columns.tolist(), 5, 2)

TOPIC 0

stay davy crockett ranch (43.4%)

stay hotel new york (5.6%)

wait long ride really (3.9%)

second hand smoke queue (3.2%)

davy crockett ranch drive (3.2%)

TOPIC 1

wait 30 minute ride (46.2%)

ride close technical difficulty (7.4%)

stand long period time (4.9%)

leave bitter taste mouth (4.6%)

30 minute ride tell (3.4%)



In [30]:
from sklearn.decomposition import LatentDirichletAllocation

# Report Results for LDA


def get_lda_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5, n_components: int = 3):
    lda = LatentDirichletAllocation(n_components=n_components)
    W = lda.fit_transform(X)
    H = lda.components_
    for topic, vector in enumerate(H):
        print(f"TOPIC {topic}\n")
        total = vector.sum()
        top_scores = vector.argsort()[::-1][:num_top_tokens]
        token_names = list(map(lambda idx: feature_names[idx], top_scores))
        strengths = list(map(lambda idx: vector[idx] / total, top_scores))
    
        for strength, token_name in zip(strengths, token_names):
            print(f"{token_name} ({round(strength * 100, 3)}%)\n")
        print(f"=" * 50)

In [31]:
get_lda_top_tf_idf_tokens_for_topic(H, data.columns.tolist(), 5, 3)

TOPIC 0

wait 30 minute ride (0.373%)

visit 10 year ago (0.282%)

visit world florida time (0.275%)

ride long wait time (0.268%)

mr toad wild ride (0.254%)

TOPIC 1

stay hotel new york (0.327%)

indiana jones roller coaster (0.239%)

tea coffee make facility (0.225%)

pirate caribbean haunted mansion (0.224%)

long queue fast ss (0.222%)

TOPIC 2

stay davy crockett ranch (0.376%)

stay new york hotel (0.35%)

stay santa fe hotel (0.328%)

wait time ride hour (0.327%)

wait hour minute ride (0.261%)



In [32]:
branch_list = list(df.Branch.unique())

for branch in branch_list:
    print(f'               {branch}:')
    branch_lemma = list(df.loc[df['Branch']==branch, 'lemma'].values)
    X = vectorizer.fit_transform(branch_lemma)
    branch_data = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
    get_top_tf_idf_tokens_for_topic(H, data.columns.tolist(), 4, 3)

               Disneyland_HongKong:
TOPIC 0

45 minute wait turn (74.7%)

40 50 minute ride (14.7%)

20 minute wait time (10.6%)

500 ml bottle water (0.0%)

TOPIC 1

10 15 minute ride (54.0%)

30 min wait long (17.8%)

10 minute wait hour (10.2%)

30 minute fast ss (9.5%)

TOPIC 2

40 60 minute wait (100.0%)

10 15 minute queue (0.0%)

10 15 minute wait (0.0%)

45 minute meet character (0.0%)

               Disneyland_California:
TOPIC 0

bad customer service experience (73.5%)

30 minute fast ss (7.4%)

drink seriously food drink (5.0%)

attraction close pirate caribbean (4.5%)

TOPIC 1

design hard navigate eat (82.8%)

bring lunch drink save (9.6%)

clean regularly cast member (7.6%)

car ride fornia adventure (0.0%)

TOPIC 2

enjoy day really enjoy (51.9%)

day queue hour ride (13.2%)

day fornia adventure day (6.7%)

cast member city hall (6.6%)

               Disneyland_Paris:
TOPIC 0

like feed time zoo (57.4%)

hour waste money time (4.0%)

average wait time 90 (3.9%)

long 

In [33]:
branch_list

['Disneyland_HongKong', 'Disneyland_California', 'Disneyland_Paris']

In [34]:
branch_list = list(df.Branch.unique())

for branch in branch_list:
    print(f'               {branch}:')
    branch_lemma = list(df.loc[df['Branch']==branch, 'lemma'].values)
    X = vectorizer.fit_transform(branch_lemma)
    branch_data = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
    get_lda_top_tf_idf_tokens_for_topic(H, data.columns.tolist(), 4, 3)

               Disneyland_HongKong:
TOPIC 0

40 60 minute wait (3.003%)

10 15 minute queue (2.726%)

500 ml bottle water (2.713%)

40 minute wait manage (2.101%)

TOPIC 1

10 15 minute ride (3.656%)

40 min fast ss (2.391%)

12 year ago change (2.391%)

40 minute fast food (2.391%)

TOPIC 2

45 minute wait turn (5.537%)

50 minute ride pirate (3.251%)

40 50 minute ride (2.992%)

30 60 min wait (2.274%)

               Disneyland_California:
TOPIC 0

bad customer service experience (1.077%)

enjoy day really enjoy (0.975%)

eat hakuna matata restaurant (0.803%)

disappoint number ride close (0.772%)

TOPIC 1

design hard navigate eat (1.003%)

enjoy stand line hour (0.913%)

download app tell wait (0.835%)

20 year ago visit (0.741%)

TOPIC 2

buggy hire 15 euro (0.882%)

90 time queue ride (0.75%)

burger chip meal come (0.71%)

dread like school holiday (0.71%)

               Disneyland_Paris:
TOPIC 0

like theme like flag (0.713%)

limit number entrance day (0.619%)

long wait rid

In [35]:
def get_top_documents_for_each_topic(num_docs: int = 5, documents=df['original_review'].to_list(), W=nmf.fit_transform(X)):
    sorted_docs = W.argsort(axis=0)[::-1]
    top_docs = sorted_docs[:num_docs].T
    per_document_totals = W.sum(axis=1)
    for topic, top_documents_for_topic in enumerate(top_docs):
        print(f"                                         Topic {topic}:")
        for doc in top_documents_for_topic:
            score = W[doc][topic]
            percent_about_topic = round(score / per_document_totals[doc] * 100, 2)
            print(f"{percent_about_topic}%", documents[doc])
            print("=" * 100)

In [36]:
get_top_documents_for_each_topic(10)

                                         Topic 0:
100.0% Overpriced. Rude staff And no place to sit. Ill Never go to Disneyland again. The beach is So Much better! Disneyland is Awful! Everything is DO EXPENSIVE. Old run down park that needs to be Completely renovated with all the Millions they are profiting Everyday. Terrible Amusement park. Awful Staff! Awful pricing on Everything! NEGATIVE STARS.
100.0% Visited May 16, 2015 and found the over all professionalism at this park much different from Disney World. Many cast members were seen chatting with one another and not greeting guests or helpful. Restaurant lost our reservation. Park at night is extremely dark. I have regularly gone to Disney World since it opened in 1971 and all of us in this group have even been to Disney Paris. All 5 of us noticed the level of service and professionalism at this park was lacking. ( ages 55  23 ) Won't go again. For $155 for a one day Park hopper you should expect Disney service. It was not the ha

In [37]:
for branch in branch_list:
    print(f"                                           {branch}:")
    branch_reviews = df[df['Branch']==branch].reset_index(drop=True)
    documents = branch_reviews['original_review']
    X, terms = vectorizer.fit_transform(branch_reviews['lemma']), vectorizer.get_feature_names()
    nmf = NMF(n_components=3)
    W = nmf.fit_transform(X)
    H = nmf.components_
    sorted_docs = W.argsort(axis=0)[::-1]
    top_docs = sorted_docs[:10].T
    per_document_totals = W.sum(axis=1)
    for topic in range(len(top_docs)):
        print(f"     Topic {topic}:")
        for doc in top_docs[topic]:
            print(documents[doc])
            print("=" * 100)

                                           Disneyland_HongKong:
     Topic 0:
After visiting Disneyland Paris and The Universal Studios in Singapore, Disneyland Hong Kong was not so great. The attractions are mainly for kids and even the 'most extreme' are never so exciting. Some of them were under maintenance and the 'soldier parachute' was still being tested at 12.00pm which is unacceptable for those who pay a 1 day ticket.The worst was Star Wars attraction which is basically nothing compared to the Transformers attraction at the Universal Studios. They only used some projections and lights, no high speed, no 3D or 4D effects!If you want to take a picture with Disney characters you have to queue for longtime..why don't they hire more staff to represent Disney characters and let them walk around so everybody can have the chance to take a pic?? At Disneyland Paris we took a lot of pics and the characters walked around at any time! I love Disney world but I think this Disneyland should 

     Topic 0:
Just got home from a long weekend in Anaheim. Have been to Disney several times but this was not one of my favorite trips. Several rides in the park are closed for refurbishment   Indiana Jones, haunted mansion, and a large part of fantasyland. These are my two favorite rides!!!!! City hall is also under reconstruction which means giant tarps over the facade. They moved the bakery with the heavenly cinnamon rolls   it just isn't the same. The heat made everyone cranky and short tempered   including cast members. I would feel bad for anyone visiting for the first time   not the magical Disneyland experience we've come to expect. On a brighter note   California adventure was awesome!!!!!!
If you have been to Florida then this is a come down. Notably smaller in all ways and more cramped than WDW. Many of the rides weren't working on day visited so couldn't ride Indiana Jones or Haunted Mansion or the Matterhorn bobsleds as either didn't work all day or led to very long queue

     Topic 0:
First time at Eurodisney and disappointed with the lack of appearance of Disney characters. We were there for 5 days but could not get on a third of the attractions as either shut or queues too long   up to 1.5hrs. Food very expensive, visited Cafe Mickey and table was not very clean. Toilets were not very clean in the parks and smelly. Not a magical experience, Florida is far superior. Would not visit here again, there is no atmosphere. Stayed at Davy Crockett Ranch, accomodation was very good.
We had such high expectations of the park and were really looking toward to our trip but were bitterly disappointed. Our boys are 4,6 and 13 and were all excited to stay for 3days. We stayed in the Davy crockett ranch which was good but that's where the magic ended. The park was way too busy and nothing like the world you imagine. Poor queuing system for rides (they need to adopt Legolands Q bot system), shocking toilets, rude people Q jumping and smoking in front of children. NO 